In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)

    from section_utils import SectionUtilities
    su = SectionUtilities(s=s, ha=ha, wsu=wsu, cu=cu, verbose=False)

    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    lru.build_isheader_logistic_regression_elements()
    lru.build_pos_logistic_regression_elements()

    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

In [5]:

%run ../load_magic/dataframes.py
from urllib.parse import urlparse, parse_qs
from urllib.error import HTTPError, URLError
from IPython.display import clear_output
from datetime import datetime
print(f'Last run on {datetime.now()}')

Last run on 2022-07-25 10:12:27.330749



----

In [16]:

file_path = '../data/html/indeed_email.html'
page_soup = wsu.get_page_soup(file_path)
css_selector = 'body > table > tbody > tr > td > a > table > tbody > tr > td > a'
link_soups_list = page_soup.select(css_selector)
files_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    file_node_dict, files_list = su.load_indeed_posting_url(viewjob_url=url_str, files_list=files_list, verbose=True)
print(f'Fileing {len(files_list)} postings complete')


                MATCH (fn:FileNames {file_name: "Qlikview_Software_Engineer_New_York_NY_Indeed_com.html"})
                SET fn.posting_url = "https://www.indeed.com/pagead/clk/dl?mo=r&ad=-6NYlbfkN0Cnp95dEus6hpwNZzYO14T2JYXzT_gSfUX9Wy6PXUfQDWuhabOeV2rgtvG_80Q8RVgTppAOBERaGmqU2Tjy6ADYj_WI-6p4NU4Vw6gZa9zojHPNlmiT5p8yiTokl6cCT3ntNN2h-8sH2ZUgmnv1ySdUXBveiNqkHi2BQr7-6FIUae7cwW_SoQkOAhJy2y3AEaVjR6w83Ej5y4MHCMGC_1Hl2v3lc4_muAffn5dv7UIkggQ7LC_JAdlj2_fJnAHtF6zwN4U2OZ2qp7LdnIOMp_kN0_1xYFp7fxJ-cKIHff5hnM99Tfr7aH_KNseWJOuy4yqRD7mibxYiE74lHxN-mt9ilSNTeOZxZaT-rFbBCRc1MAfHqsEKdFvB3UM9qwPxI6wl_3JKHOvyjtdPnSHvjMYXoOrqQvKocW_zI3Hk1m9Jo4eeU0a-1xVPlRlFBZ4lP93wvtO-CLyyfIwS75pU0Xp_4IpqyFHrc2QXgQ75CqcWRY78O5pBUpdIfLCtxm_5lzI%3D&xkcb=SoAU-_M3bThuMc7btp0LbzkdCdPP&p=0&jsa=3954&rjs=1&tmtk=1g8jkilhbq5ld800&gdfvj=1&alid=62cdb0067c78606171e5793c&fvj=1"
                RETURN fn;
[{0: 'Qlikview_Software_Engineer_New_York_NY_Indeed_com.html'}, {0: 'https://www.indeed.com/pagead/clk/dl?mo=r&ad=-6NYlbfkN0Cnp95dEus6h

In [17]:

cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
files_list

[]